In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,export_graphviz
import graphviz
import dtreeviz
import pandas as pd
import numpy as np
import pickle
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn import tree
from pypinyin import pinyin, lazy_pinyin, Style
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
# 复制一份并改写类别
def duplicate_and_rename(df, column_name, new_tz_name):

    # 复制一份数据
    duplicated_df = df.copy()
    # 修改复制的数据的NEW_TZ名称
    duplicated_df[column_name] = new_tz_name
    # 返回包含原数据和修改后的数据的DataFrame
    return pd.concat([df, duplicated_df])

In [3]:
# autogluon保存路径
model_path = r"F:\cache_data\zone_ana\qz\modle\autogluon_type"
# 检查路径是否存在，否则便创建
if not os.path.exists(model_path):
    os.makedirs(model_path)
else:
    print("文件夹已存在")

文件夹已存在


In [4]:
dataset = pd.read_csv(r"F:\cache_data\zone_ana\qz\train_data\soil_type_train_point.csv")

In [5]:
def balance_sample_groups(dataset, group_columns, min_samples=20):
    """
    对数据集中指定列组合的样本数进行平衡处理
    
    参数:
    dataset: pd.DataFrame, 输入的数据集
    group_columns: str或list, 用于分组的列名，可以是单个列名或多个列名的列表
    min_samples: int, 每个组合的最小样本数，默认为20
    
    返回:
    pd.DataFrame: 平衡后的数据集
    """
    # 确保group_columns是列表
    if isinstance(group_columns, str):
        group_columns = [group_columns]
        
    # 复制数据集以避免修改原始数据
    dataset = dataset.copy()
    
    # 统计每个组合的样本数
    group_counts = dataset.groupby(group_columns).size().reset_index(name='count')
    # 找出样本数少于min_samples的组合
    low_count_groups = group_counts[group_counts['count'] < min_samples]
    
    # 记录处理前的样本总数
    original_count = len(dataset)
    
    # 对每个样本数不足的组合进行处理
    for _, group in low_count_groups.iterrows():
        # 构建过滤条件
        filter_cond = True
        for col in group_columns:
            filter_cond = filter_cond & (dataset[col] == group[col])
            
        # 获取当前组合的所有样本
        group_samples = dataset[filter_cond]
        current_count = len(group_samples)
        
        # 计算需要复制的样本数
        samples_needed = min_samples - current_count
        
        # 根据当前样本数决定复制方式
        if current_count == 1:
            samples_to_add = pd.concat([group_samples] * samples_needed)
        else:
            samples_to_add = group_samples.sample(n=samples_needed, replace=True)
            
        # 将复制的样本添加到数据集中
        dataset = pd.concat([dataset, samples_to_add], ignore_index=True)
    
    print(f"处理前样本总数: {original_count}")
    print(f"处理后样本总数: {len(dataset)}")
    
    return dataset

# 使用示例:
# 单个因素
# dataset = balance_sample_groups(dataset, 'NEW_TZ')
# 多个因素
dataset = balance_sample_groups(dataset, ['NEW_TZ', 'MZMC','DL'])


处理前样本总数: 14007
处理后样本总数: 16164


In [6]:
dataset.shape

(16164, 99)

In [7]:
dataset["TZ_label"] = dataset.NEW_TZ.astype("category").cat.codes

In [8]:
dataset['dl'] = dataset['dl'].astype('category')
dataset['dz'] = dataset['dz'].astype('category')
dataset['slopepostion'] = dataset['slopepostion'].astype('category')
dataset['TZ_label'] = dataset['TZ_label'].astype('category')

In [9]:
result = dataset.groupby('TZ_label', observed=True)["NEW_TZ"].apply(lambda x: list(x.unique())).to_dict()
for one_type in result:
    result[one_type] = result[one_type][0]
print(result)

{0: '中层壤质灰潮土', 1: '中层壤质红色石灰土', 2: '中层壤质黄色石灰土', 3: '中层壤质黑色石灰土', 4: '中层暗泥质黄壤', 5: '中层泥质黄壤', 6: '中层灰泥质黄壤', 7: '中层灰泥质黄棕壤', 8: '中层砂泥质黄壤', 9: '中层砂质灰潮土', 10: '中层硅质黄壤', 11: '中层黏质黄色石灰土', 12: '厚层壤质灰潮土', 13: '厚层泥质黄壤', 14: '厚层灰泥质黄壤', 15: '厚层砂质灰潮土', 16: '厚层硅质黄壤', 17: '浅石灰泥田', 18: '渗潮泥田', 19: '渗石灰泥田', 20: '渗鳝泥田', 21: '潮泥田', 22: '石灰泥田', 23: '腐中层壤质灰潮土', 24: '腐中层壤质红色石灰土', 25: '腐中层壤质黄色石灰土', 26: '腐中层壤质黑色石灰土', 27: '腐中层暗泥质黄壤', 28: '腐中层泥质黄壤', 29: '腐中层灰泥质黄壤', 30: '腐中层砂泥质黄壤', 31: '腐中层砂质灰潮土', 32: '腐中层砂质黄色石灰土', 33: '腐中层硅质黄壤', 34: '腐中层黏质黄色石灰土', 35: '腐厚层泥质黄壤', 36: '腐薄层壤质红色石灰土', 37: '腐薄层壤质黄色石灰土', 38: '腐薄层壤质黑色石灰土', 39: '腐薄层暗泥质黄壤性土', 40: '腐薄层暗泥质黄棕壤', 41: '腐薄层暗泥质黄棕壤性土', 42: '腐薄层泥质黄壤', 43: '腐薄层泥质黄壤性土', 44: '腐薄层泥质黄棕壤', 45: '腐薄层泥质黄棕壤性土', 46: '腐薄层灰泥质黄壤性土', 47: '腐薄层灰泥质黄棕壤', 48: '腐薄层灰泥质黄棕壤性土', 49: '腐薄层砂泥质黄壤性土', 50: '腐薄层砂泥质黄棕壤', 51: '腐薄层砂泥质黄棕壤性土', 52: '腐薄层砂质黄色石灰土', 53: '腐薄层硅质黄壤', 54: '腐薄层硅质黄壤性土', 55: '腐薄层硅质黄棕壤', 56: '腐薄层硅质黄棕壤性土', 57: '腐薄层黏质黄色石灰土', 58: '薄层壤质红色石灰土', 59: '薄层壤质黄色石灰土', 60: '薄层壤质黑色石灰土', 61: '薄层暗泥质黄棕壤', 62: '薄层泥质黄

In [10]:
# 保存映射关系
with open(r'D:\worker_code\Terrain_Test\data\soil_dict_20250219_qz.json', 'w') as f:
    json.dump(result, f)

In [11]:
# 查看地类类别
tdlylx_df = pd.DataFrame(dataset['DLLB'].value_counts())
tdlylx_df

,count
DLLB,
自然土,8385
非自然土,6202
水稻土,1577


In [12]:
# 查看地类类别
dl_df = pd.DataFrame(dataset['DL'].value_counts())
dl_df

,count
DL,
林地,8325
旱地,4501
水田,1417
种植园用地,1101
其他土地,480
水浇地,160
湿地,120
草地,60


In [13]:
# 查看母质类别
mz_df = pd.DataFrame(dataset['MZMC'].value_counts())
mz_df

,count
MZMC,
灰岩,8186
白云岩,3452
砂岩,1498
泥(页)岩,1296
河流冲积物,572
粘土岩,299
页岩,254
玄武岩,246
砂页岩,221


### 砂岩土种

In [14]:
# 筛选砂岩类水稻土数据
sy_sdt_data = dataset[(dataset['DLLB']=='水稻土')
                      &(dataset['MZMC'] == '砂岩') ]
pd.unique(sy_sdt_data['NEW_TZ']),#pd.unique(sy_sdt_data['NEW_TS'])

(array(['黄白粉泥田', '黄砂泥田', '黄浅白粉泥田', '重漂砂泥田', '黄青砂泥田', '黄浅砂泥田'], dtype=object),)

In [15]:
# 筛砂岩选非自然土数据  
sy_fzrt_data = dataset[(dataset['DLLB']=='非自然土') 
                       & ((dataset['MZMC'] == '砂岩'))]
pd.unique(sy_fzrt_data['NEW_TZ']),#pd.unique(sy_fzrt_data['NEW_TS'])

(array(['厚层硅质黄壤', '中层硅质黄壤', '薄层硅质黄壤', '薄层硅质黄壤性土', '薄层硅质黄棕壤', '薄层硅质黄棕壤性土'],
       dtype=object),)

In [16]:
# 筛选自然土数据
sy_zrt_data = dataset[(dataset['DLLB']=='自然土')
                   & ((dataset['MZMC'] == '砂岩') )]
pd.unique(sy_zrt_data['NEW_TZ']),#pd.unique(sy_zrt_data['NEW_TS'])

(array(['腐中层硅质黄壤', '腐薄层硅质黄壤', '腐薄层硅质黄壤性土', '薄层硅质黄棕壤', '腐薄层硅质黄棕壤性土',
        '腐薄层硅质黄棕壤'], dtype=object),)

### 碳酸岩土种

In [17]:
# 筛选碳酸岩水稻土数据
tsy_sdt_data = dataset[(dataset['DLLB']=='水稻土')
                      & ((dataset['MZMC'] == '白云岩')|(dataset['MZMC'] == '灰岩'))]
pd.unique(tsy_sdt_data['NEW_TZ']),#pd.unique(tsy_sdt_data['NEW_TS']),

(array(['石灰泥田', '青石灰泥田', '渗石灰泥田', '轻漂灰泥田', '浅石灰泥田', '重漂灰泥田'], dtype=object),)

In [18]:
# 筛选碳酸岩选非自然土数据
tsy_fzrt_data = dataset[(dataset['DLLB']=='非自然土')
                       & ((dataset['MZMC'] == '白云岩')|(dataset['MZMC'] == '灰岩'))]

pd.unique(tsy_fzrt_data['NEW_TZ']),#pd.unique(tsy_fzrt_data['NEW_TS']),

(array(['中层灰泥质黄壤', '薄层灰泥质黄壤', '中层壤质黄色石灰土', '薄层壤质黄色石灰土', '中层壤质黑色石灰土',
        '厚层灰泥质黄壤', '薄层黏质黄色石灰土', '中层黏质黄色石灰土', '薄层灰泥质黄壤性土', '中层壤质红色石灰土',
        '薄层壤质黑色石灰土', '薄层灰泥质黄棕壤', '薄层壤质红色石灰土', '薄层灰泥质黄棕壤性土', '中层灰泥质黄棕壤'],
       dtype=object),)

In [19]:
# 筛选碳酸岩自然土数据

tsy_zrt_data = dataset[(dataset['DLLB']=='自然土')
                       & ((dataset['MZMC'] == '白云岩')|(dataset['MZMC'] == '灰岩'))]
pd.unique(tsy_zrt_data['NEW_TZ']),#pd.unique(tsy_zrt_data['NEW_TS']),

(array(['腐中层灰泥质黄壤', '腐中层壤质黄色石灰土', '腐薄层壤质黄色石灰土', '腐中层壤质黑色石灰土', '腐薄层壤质黑色石灰土',
        '腐薄层砂质黄色石灰土', '腐中层砂质黄色石灰土', '腐薄层灰泥质黄壤性土', '腐中层壤质红色石灰土',
        '腐薄层壤质红色石灰土', '薄层灰泥质黄棕壤', '腐薄层灰泥质黄棕壤性土', '腐薄层灰泥质黄棕壤', '腐薄层黏质黄色石灰土',
        '腐中层黏质黄色石灰土'], dtype=object),)

### 泥(页)岩

In [20]:
# 筛选泥(页)岩水稻土数据
nyy_sdt_data = dataset[(dataset['DLLB']=='水稻土')
                       & ((dataset['MZMC'] == '泥(页)岩') | (dataset['MZMC'] == '页岩') | (dataset['MZMC'] == '粘土岩') | (dataset['MZMC'] == '板岩') | (dataset['MZMC'] == '泥岩'))]
pd.unique(nyy_sdt_data['NEW_TZ']),#pd.unique(nyy_sdt_data['NEW_TS']),

(array(['黄渗鳝泥田', '黄鳝泥田', '黄青鳝泥田', '轻漂鳝泥田', '黄浅鳝泥田', '渗鳝泥田'], dtype=object),)

In [21]:
# 筛选泥(页)岩非自然土数据
nyy_fzrt_data = dataset[(dataset['DLLB']=='非自然土')
                       & ((dataset['MZMC'] == '泥(页)岩') | (dataset['MZMC'] == '页岩') | (dataset['MZMC'] == '粘土岩') | (dataset['MZMC'] == '板岩') | (dataset['MZMC'] == '泥岩'))]
pd.unique(nyy_fzrt_data['NEW_TZ']),#pd.unique(nyy_fzrt_data['NEW_TS']),

(array(['中层泥质黄壤', '厚层泥质黄壤', '薄层泥质黄壤性土', '薄层泥质黄棕壤', '薄层砂泥质黄棕壤'],
       dtype=object),)

In [22]:
# 筛选泥(页)岩自然土数据    
nyy_zrt_data = dataset[(dataset['DLLB']=='自然土')
                       & ((dataset['MZMC'] == '泥(页)岩') | (dataset['MZMC'] == '页岩') | (dataset['MZMC'] == '粘土岩') | (dataset['MZMC'] == '板岩') | (dataset['MZMC'] == '泥岩'))]
pd.unique(nyy_zrt_data['NEW_TZ']),#pd.unique(nyy_zrt_data['NEW_TS']),

(array(['腐中层泥质黄壤', '腐薄层泥质黄壤', '腐厚层泥质黄壤', '腐薄层泥质黄棕壤', '腐薄层泥质黄壤性土',
        '薄层泥质黄棕壤', '腐薄层泥质黄棕壤性土'], dtype=object),)

### 河流冲积物

In [24]:
# 筛选河流冲积物水稻土数据
hlcjw_sdt_data = dataset[(dataset['DLLB']=='水稻土')
                       & (dataset['MZMC'] == '河流冲积物')]
pd.unique(hlcjw_sdt_data['NEW_TZ']),#pd.unique(hlcjw_sdt_data['NEW_TS']),

(array(['潮泥田', '青潮泥田', '渗潮泥田', '轻漂潮泥田'], dtype=object),)

In [25]:
# 筛选河流冲积物非自然土数据
hlcjw_fzrt_data = dataset[(dataset['DLLB']=='非自然土')
                       & (dataset['MZMC'] == '河流冲积物')]
pd.unique(hlcjw_fzrt_data['NEW_TZ']),#pd.unique(hlcjw_fzrt_data['NEW_TS']),

(array(['中层壤质灰潮土', '厚层砂质灰潮土', '厚层壤质灰潮土', '中层砂质灰潮土'], dtype=object),)

In [26]:
# 筛选河流冲积物自然土数据
hlcjw_zrt_data = dataset[(dataset['DLLB']=='自然土')
                       & (dataset['MZMC'] == '河流冲积物')]
pd.unique(hlcjw_zrt_data['NEW_TZ']),#pd.unique(hlcjw_zrt_data['NEW_TS']),


(array(['腐中层壤质灰潮土', '腐中层砂质灰潮土', '厚层砂质灰潮土', '中层壤质灰潮土'], dtype=object),)

### 砂页岩

In [27]:
# 筛选砂页岩水稻土数据
syy_sdt_data = dataset[(dataset['DLLB']=='水稻土')
                       & ((dataset['MZMC'] == '砂页岩') | (dataset['MZMC'] == '砾岩'))]
pd.unique(syy_sdt_data['NEW_TZ']),#pd.unique(syy_sdt_data['NEW_TS']),

(array(['黄砂泥田'], dtype=object),)

In [28]:
# 复制一份并改写类别
syy_sdt_data = duplicate_and_rename(syy_sdt_data, 'NEW_TZ', '黄浅砂泥田')
pd.unique(syy_sdt_data['NEW_TZ']),#pd.unique(syy_sdt_data['NEW_TS']),


(array(['黄砂泥田', '黄浅砂泥田'], dtype=object),)

In [29]:
# 筛选砂页岩非自然土数据
syy_fzrt_data = dataset[(dataset['DLLB']=='非自然土')
                       & ((dataset['MZMC'] == '砂页岩') | (dataset['MZMC'] == '砾岩'))]
pd.unique(syy_fzrt_data['NEW_TZ']),#pd.unique(syy_fzrt_data['NEW_TS']),

(array(['中层砂泥质黄壤', '薄层砂泥质黄棕壤', '薄层砂泥质黄棕壤性土'], dtype=object),)

In [30]:
# 筛选砂页岩物自然土数据
syy_zrt_data = dataset[(dataset['DLLB']=='自然土')
                       & ((dataset['MZMC'] == '砂页岩') | (dataset['MZMC'] == '砾岩'))]
pd.unique(syy_zrt_data['NEW_TZ']),#pd.unique(syy_zrt_data['NEW_TS']),


(array(['腐中层砂泥质黄壤', '腐薄层砂泥质黄棕壤', '腐薄层砂泥质黄壤性土', '薄层砂泥质黄棕壤', '腐薄层砂泥质黄棕壤性土'],
       dtype=object),)

# 玄武岩

In [31]:
# 筛选玄武岩水稻土数据
xw_sdt_data = dataset[(dataset['DLLB']=='水稻土')
                       & ((dataset['MZMC'] == '玄武岩'))]
pd.unique(xw_sdt_data['NEW_TZ']),#pd.unique(xw_sdt_data['NEW_TS']),

(array(['黄暗泥田'], dtype=object),)

In [32]:
# 复制一份并改写类别
xw_sdt_data = duplicate_and_rename(xw_sdt_data, 'NEW_TZ', '黄渗暗泥田')
pd.unique(xw_sdt_data['NEW_TZ']),

(array(['黄暗泥田', '黄渗暗泥田'], dtype=object),)

In [35]:
# 筛选玄武岩自然土数据
xw_zrt_data = dataset[(dataset['DLLB']=='自然土')
                       & ((dataset['MZMC'] == '玄武岩'))]
pd.unique(xw_zrt_data['NEW_TZ']),#pd.unique(xw_zrt_data['NEW_TS']),


(array(['腐薄层暗泥质黄壤性土', '腐中层暗泥质黄壤', '腐薄层暗泥质黄棕壤', '腐薄层暗泥质黄棕壤性土', '薄层暗泥质黄棕壤'],
       dtype=object),)

In [34]:
# 筛选玄武岩非自然土数据
xw_fzrt_data = dataset[(dataset['DLLB']=='非自然土')
                       & ((dataset['MZMC'] == '玄武岩'))]
pd.unique(xw_fzrt_data['NEW_TZ']),#pd.unique(xw_fzrt_data['NEW_TS']),

(array(['中层暗泥质黄壤', '薄层暗泥质黄棕壤'], dtype=object),)

In [36]:
features_use = ['aspect', 'carbonate', 'channelnetworkbaselevel', 'channelnetworkdistance', 'clay_minerals', 'contrast', 
                'convergenceindex', 'correlation', 'dem', 'dissimilarity', 'dl', 'dz', 'entropy', 'etp22_3', 'etp22_mean', 
                'evi', 'ferrous_minerals', 'hillshade', 'homogeneity', 'lat', 'lon', 'lsfactor', 'lswi', 'mean', 'mndwi', 
                'mrrtf', 'mrvbf', 'ndmi', 'ndvi', 'ndwi', 'night22_', 'pc1', 'pc2', 'plancurvature', 'pre22_3', 'pre22_mean', 
                'profilecurvature', 'relativeslopeposition', 'rock_outcrop', 'savi', 'secondmoment', 'slope', 'slopepostion', 
                'terrainruggednessindex', 'tmp22_3', 'tmp22_mean', 'topographicwetnessindex', 'totalcatchmentarea', 'valleydepth',
                'vari', 'variance']

In [37]:

sdt_features = features_use
fzrt_features = features_use
zrt_features = features_use
target = "TZ_label"

In [43]:
# 如果不相等则需要重新合并数据后重新计算TZ_label
calc_df = pd.concat([sy_sdt_data,sy_fzrt_data,sy_zrt_data,tsy_sdt_data,tsy_fzrt_data,tsy_zrt_data,nyy_sdt_data,nyy_fzrt_data,nyy_zrt_data,hlcjw_sdt_data,hlcjw_fzrt_data,hlcjw_zrt_data,syy_sdt_data,syy_fzrt_data,syy_zrt_data,xw_sdt_data,xw_zrt_data,xw_fzrt_data])

calc_df["TZ_label"] = calc_df.NEW_TZ.astype("category").cat.codes
result = calc_df.groupby('TZ_label', observed=True)["NEW_TZ"].apply(lambda x: list(x.unique())).to_dict()
for one_type in result:
    result[one_type] = result[one_type][0]
print(result)
# 保存映射关系
with open(r'D:\worker_code\Terrain_Test\data\soil_dict_20250219_qz.json', 'w') as f:
    json.dump(result, f)

{0: '中层壤质灰潮土', 1: '中层壤质红色石灰土', 2: '中层壤质黄色石灰土', 3: '中层壤质黑色石灰土', 4: '中层暗泥质黄壤', 5: '中层泥质黄壤', 6: '中层灰泥质黄壤', 7: '中层灰泥质黄棕壤', 8: '中层砂泥质黄壤', 9: '中层砂质灰潮土', 10: '中层硅质黄壤', 11: '中层黏质黄色石灰土', 12: '厚层壤质灰潮土', 13: '厚层泥质黄壤', 14: '厚层灰泥质黄壤', 15: '厚层砂质灰潮土', 16: '厚层硅质黄壤', 17: '浅石灰泥田', 18: '渗潮泥田', 19: '渗石灰泥田', 20: '渗鳝泥田', 21: '潮泥田', 22: '石灰泥田', 23: '腐中层壤质灰潮土', 24: '腐中层壤质红色石灰土', 25: '腐中层壤质黄色石灰土', 26: '腐中层壤质黑色石灰土', 27: '腐中层暗泥质黄壤', 28: '腐中层泥质黄壤', 29: '腐中层灰泥质黄壤', 30: '腐中层砂泥质黄壤', 31: '腐中层砂质灰潮土', 32: '腐中层砂质黄色石灰土', 33: '腐中层硅质黄壤', 34: '腐中层黏质黄色石灰土', 35: '腐厚层泥质黄壤', 36: '腐薄层壤质红色石灰土', 37: '腐薄层壤质黄色石灰土', 38: '腐薄层壤质黑色石灰土', 39: '腐薄层暗泥质黄壤性土', 40: '腐薄层暗泥质黄棕壤', 41: '腐薄层暗泥质黄棕壤性土', 42: '腐薄层泥质黄壤', 43: '腐薄层泥质黄壤性土', 44: '腐薄层泥质黄棕壤', 45: '腐薄层泥质黄棕壤性土', 46: '腐薄层灰泥质黄壤性土', 47: '腐薄层灰泥质黄棕壤', 48: '腐薄层灰泥质黄棕壤性土', 49: '腐薄层砂泥质黄壤性土', 50: '腐薄层砂泥质黄棕壤', 51: '腐薄层砂泥质黄棕壤性土', 52: '腐薄层砂质黄色石灰土', 53: '腐薄层硅质黄壤', 54: '腐薄层硅质黄壤性土', 55: '腐薄层硅质黄棕壤', 56: '腐薄层硅质黄棕壤性土', 57: '腐薄层黏质黄色石灰土', 58: '薄层壤质红色石灰土', 59: '薄层壤质黄色石灰土', 60: '薄层壤质黑色石灰土', 61: '薄层暗泥质黄棕壤', 62: '薄层泥质黄

In [ ]:
# 逐个数据更新TZ_label，根据result
for one_data in [sy_sdt_data,sy_fzrt_data,sy_zrt_data,tsy_sdt_data,tsy_fzrt_data,tsy_zrt_data,nyy_sdt_data,nyy_fzrt_data,nyy_zrt_data,hlcjw_sdt_data,hlcjw_fzrt_data,hlcjw_zrt_data,syy_sdt_data,syy_fzrt_data,syy_zrt_data,xw_sdt_data,xw_zrt_data,xw_fzrt_data]:
    one_data['TZ_label'] = one_data['NEW_TZ'].map({v: k for k, v in result.items()})

### 砂岩数据集

In [55]:
# 获取sdt数据集
sy_sdt_data = sy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
sy_fzrt_data = sy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
sy_zrt_data = sy_zrt_data[zrt_features+[target]]

### 碳酸岩数据集

In [56]:
# 获取sdt数据集
tsy_sdt_data = tsy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
tsy_fzrt_data = tsy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
tsy_zrt_data = tsy_zrt_data[zrt_features+[target]]

### 泥(页)岩数据集

In [57]:
# 获取sdt数据集
nyy_sdt_data = nyy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
nyy_fzrt_data = nyy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
nyy_zrt_data = nyy_zrt_data[zrt_features+[target]]

### 河流冲积物数据集

In [58]:
# 获取sdt数据集
hlcjw_sdt_data = hlcjw_sdt_data[sdt_features+[target]]
# 获取非自然土
hlcjw_fzrt_data = hlcjw_fzrt_data[fzrt_features+[target]]
# 获取自然土
hlcjw_zrt_data = hlcjw_zrt_data[zrt_features+[target]]

### 砂页岩数据集

In [59]:
# 获取sdt数据集
syy_sdt_data = syy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
syy_fzrt_data = syy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
syy_zrt_data = syy_zrt_data[zrt_features+[target]]

### 玄武岩数据集

In [60]:
# 获取水稻土数据集
xw_sdt_data = xw_sdt_data[sdt_features+[target]]
# 获取峨嵋山玄武岩数据集
xw_zrt_data = xw_zrt_data[zrt_features+[target]]
# 获取峨嵋山玄武岩非自然土数据集
xw_fzrt_data = xw_fzrt_data[fzrt_features+[target]]

In [61]:
# 查看各个数据集的大小
print('砂岩',sy_sdt_data.shape,sy_fzrt_data.shape,sy_zrt_data.shape)
print('碳酸岩',tsy_sdt_data.shape,tsy_fzrt_data.shape,tsy_zrt_data.shape)
# print('第四系红粘土',hnt_sdt_data.shape,hnt_fzrt_data.shape,hnt_zrt_data.shape)
print('泥页岩',nyy_sdt_data.shape,nyy_fzrt_data.shape,nyy_zrt_data.shape)
# print('紫红色砂页岩',zhsyy_sdt_data.shape,zhsyy_fzrt_data.shape,zhsyy_zrt_data.shape)
print('河流冲积物',hlcjw_sdt_data.shape,hlcjw_fzrt_data.shape,hlcjw_zrt_data.shape)
print('砂页岩',syy_sdt_data.shape,syy_fzrt_data.shape,syy_zrt_data.shape)
print('玄武岩',xw_sdt_data.shape,xw_zrt_data.shape,xw_fzrt_data.shape)

# 计算总数
# total_data = pd.concat([sy_sdt_data,sy_fzrt_data,sy_zrt_data,tsy_sdt_data,tsy_fzrt_data,tsy_zrt_data,hnt_sdt_data,hnt_fzrt_data,hnt_zrt_data,nyy_sdt_data,nyy_fzrt_data,nyy_zrt_data,zhsyy_sdt_data,zhsyy_fzrt_data,zhsyy_zrt_data,hlcjw_sdt_data,hlcjw_fzrt_data,hlcjw_zrt_data,syy_sdt_data,syy_fzrt_data,syy_zrt_data,xw_sdt_data,xw_zrt_data,xw_fzrt_data])
total_data = pd.concat([sy_sdt_data,sy_fzrt_data,sy_zrt_data,tsy_sdt_data,tsy_fzrt_data,tsy_zrt_data,nyy_sdt_data,nyy_fzrt_data,nyy_zrt_data,hlcjw_sdt_data,hlcjw_fzrt_data,hlcjw_zrt_data,syy_sdt_data,syy_fzrt_data,syy_zrt_data,xw_sdt_data,xw_zrt_data,xw_fzrt_data])
total_data.shape,dataset.shape[0]


砂岩 (172, 52) (492, 52) (834, 52)
碳酸岩 (785, 52) (4662, 52) (6191, 52)
泥页岩 (356, 52) (667, 52) (966, 52)
河流冲积物 (224, 52) (259, 52) (89, 52)
砂页岩 (40, 52) (60, 52) (141, 52)
玄武岩 (40, 52) (164, 52) (62, 52)


((16204, 52), 16164)

In [62]:
# 确定标签
label = "TZ_label"
eval_metric = 'f1_weighted'
problem_type = 'multiclass'
# # 10折交叉验证
# cv_num = 10
# 初始化模型和超参数
hyperparameters={
	'NN_TORCH': {},
	'FASTAI': {},
	'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}]
}


### 模型训练

In [63]:
def train_model(train_data,label,eval_metric,hyperparameters,problem_type,model_master,model_branch,model_root):
    # 训练model
    temp_master_path =os.path.join(model_root,model_master)
    temp_branch_path = os.path.join(temp_master_path,model_branch)
    # 检查路径是否存在，否则便创建
    if not os.path.exists(temp_branch_path):
        os.makedirs(temp_branch_path)
    else:
        print("文件夹已存在")
    # 执行训练
    train_predictor = TabularPredictor(label=label,path=temp_branch_path,eval_metric=eval_metric,problem_type=problem_type).fit(train_data,hyperparameters=hyperparameters)
    train_predictor.fit_summary()
    return train_predictor.model_best


### 模型训练

In [64]:
# 砂岩
sy_sdt_predictor = train_model(sy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='sy',model_branch='sdt',model_root=model_path) 
sy_fzrt_predictor = train_model(sy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='sy',model_branch='fzrt',model_root=model_path) 
sy_zrt_predictor = train_model(sy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='sy',model_branch='zrt',model_root=model_path) 
# 碳酸岩
tsy_sdt_predictor = train_model(tsy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='tsy',model_branch='sdt',model_root=model_path) 
tsy_fzrt_predictor = train_model(tsy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='tsy',model_branch='fzrt',model_root=model_path) 
tsy_zrt_predictor = train_model(tsy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='tsy',model_branch='zrt',model_root=model_path) 
# 第四系红粘土
# hnt_sdt_predictor = train_model(hnt_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hnt',model_branch='sdt',model_root=model_path) 
# hnt_fzrt_predictor = train_model(hnt_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hnt',model_branch='fzrt',model_root=model_path) 
# hnt_zrt_predictor = train_model(hnt_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hnt',model_branch='zrt',model_root=model_path) 
# 泥页岩
nyy_sdt_predictor = train_model(nyy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nyy',model_branch='sdt',model_root=model_path) 
nyy_fzrt_predictor = train_model(nyy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nyy',model_branch='fzrt',model_root=model_path) 
nyy_zrt_predictor = train_model(nyy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nyy',model_branch='zrt',model_root=model_path) 
# 紫红色砂页岩
# zhsyy_sdt_predictor = train_model(zhsyy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='zhsyy',model_branch='sdt',model_root=model_path) 
# zhsyy_fzrt_predictor = train_model(zhsyy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='zhsyy',model_branch='fzrt',model_root=model_path) 
# zhsyy_zrt_predictor = train_model(zhsyy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='zhsyy',model_branch='zrt',model_root=model_path) 
# 河流冲积物
hlcjw_sdt_predictor = train_model(hlcjw_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hlcjw',model_branch='sdt',model_root=model_path)
hlcjw_fzrt_predictor = train_model(hlcjw_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hlcjw',model_branch='fzrt',model_root=model_path)
hlcjw_zrt_predictor = train_model(hlcjw_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hlcjw',model_branch='zrt',model_root=model_path)
# 砂页岩
syy_sdt_predictor = train_model(syy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='syy',model_branch='sdt',model_root=model_path) 
syy_fzrt_predictor = train_model(syy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='syy',model_branch='fzrt',model_root=model_path) 
syy_zrt_predictor = train_model(syy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='syy',model_branch='zrt',model_root=model_path) 
# 玄武岩
xw_sdt_predictor = train_model(xw_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='xw',model_branch='sdt',model_root=model_path) 
xw_zrt_predictor = train_model(xw_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='xw',model_branch='zrt',model_root=model_path) 
xw_fzrt_predictor = train_model(xw_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='xw',model_branch='fzrt',model_root=model_path) 

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "F:\cache_data\zone_ana\qz\modle\autogluon_type\sy\sdt"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          20
Memory Avail:       42.29 GB / 63.81 GB (66.3%)
Disk Space Avail:   442.40 GB / 

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       NeuralNetTorch   0.735678  f1_weighted       0.009998  1.230866                0.009998           1.230866            1       True          6
1  WeightedEnsemble_L2   0.735678  f1_weighted       0.010867  1.341534                0.000869           0.110667            2       True          7
2      NeuralNetFastAI   0.713571  f1_weighted       0.013197  2.068504                0.013197           2.068504            1       True          1
3       ExtraTreesGini   0.709127  f1_weighted       0.034758  0.462754                0.034758           0.462754            1       True          4
4       ExtraTreesEntr   0.686145  f1_weighted       0.023868  0.467253                0.023868           0.467253            1       True          5
5     RandomForestEntr   0.686145  f1_

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "F:\cache_data\zone_ana\qz\modle\autogluon_type\sy\fzrt"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          20
Memory Avail:       42.09 GB / 63.81 GB (66.0%)
Disk Space Avail:   442.39 GB /

Plot summary of models saved to file: F:\cache_data\zone_ana\qz\modle\autogluon_type\sy\sdtSummaryOfModels.html
*** End of fit() summary ***


	0.6927	 = Validation score   (f1_weighted)
	0.41s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.8166	 = Validation score   (f1_weighted)
	0.45s	 = Training   runtime
	0.15s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.7986	 = Validation score   (f1_weighted)
	0.48s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.7869	 = Validation score   (f1_weighted)
	0.47s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.7963	 = Validation score   (f1_weighted)
	0.49s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.8148	 = Validation score   (f1_weighted)
	2.89s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'RandomForestGini': 0.5, 'NeuralNetTorch': 0.333, 'ExtraTreesEntr': 0.167}
	0.8293	 = Validation score   (f1_weighted)
	0.1s	 = Training   runtime
	0.0s	 =

...

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.733333  f1_weighted       0.000000  0.314743                0.000000           0.079512            2       True          7
1      NeuralNetFastAI   0.733333  f1_weighted       0.000000  0.235231                0.000000           0.235231            1       True          5
2       NeuralNetTorch   0.333333  f1_weighted       0.015903  0.196007                0.015903           0.196007            1       True          6
3       ExtraTreesEntr   0.111111  f1_weighted       0.031328  0.325805                0.031328           0.325805            1       True          4
4     RandomForestGini   0.111111  f1_weighted       0.032774  0.345796                0.032774           0.345796            1       True          1
5     RandomForestEntr   0.111111  f1_

	0.934	 = Validation score   (f1_weighted)
	0.87s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7811	 = Validation score   (f1_weighted)
	0.43s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.844	 = Validation score   (f1_weighted)
	0.44s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.7811	 = Validation score   (f1_weighted)
	0.43s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.7811	 = Validation score   (f1_weighted)
	0.42s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.8935	 = Validation score   (f1_weighted)
	1.1s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetFastAI': 1.0}
	0.934	 = Validation score   (f1_weighted)
	0.16s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total 

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      NeuralNetFastAI   0.933961  f1_weighted       0.002004  0.865533                0.002004           0.865533            1       True          1
1  WeightedEnsemble_L2   0.933961  f1_weighted       0.003005  1.023132                0.001000           0.157598            2       True          7
2       NeuralNetTorch   0.893506  f1_weighted       0.011881  1.097792                0.011881           1.097792            1       True          6
3     RandomForestEntr   0.843963  f1_weighted       0.016108  0.441390                0.016108           0.441390            1       True          3
4       ExtraTreesEntr   0.781145  f1_weighted       0.032638  0.416065                0.032638           0.416065            1       True          5
5       ExtraTreesGini   0.781145  f1_

	1.0	 = Validation score   (f1_weighted)
	0.36s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestEntr ...
	1.0	 = Validation score   (f1_weighted)
	0.35s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	1.0	 = Validation score   (f1_weighted)
	0.57s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	1.0	 = Validation score   (f1_weighted)
	0.54s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
No improvement since epoch 3: early stopping
	1.0	 = Validation score   (f1_weighted)
	0.54s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	1.0	 = Validation score   (f1_weighted)
	0.61s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetFastAI': 1.0}
	1.0	 = Validation score   (f1_weighted)
	0.1s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluo

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2        1.0  f1_weighted       0.008526  0.641804                0.000000           0.100949            2       True          7
1      NeuralNetFastAI        1.0  f1_weighted       0.008526  0.540854                0.008526           0.540854            1       True          5
2       NeuralNetTorch        1.0  f1_weighted       0.013080  0.614087                0.013080           0.614087            1       True          6
3     RandomForestGini        1.0  f1_weighted       0.028488  0.363820                0.028488           0.363820            1       True          1
4     RandomForestEntr        1.0  f1_weighted       0.033426  0.347666                0.033426           0.347666            1       True          2
5       ExtraTreesGini        1.0  f1_